In [1]:
import numpy as np
import pandas as pd
import os
import spacy

In [2]:
path = os.getcwd()
train_data = "/sfc_pn_train.csv"
test_data = "/sfc_pn_test.csv"

In [3]:
datapath = os.path.split(path)[0] + os.path.join("/Data")
outputpath = os.path.split(path)[0]+ os.path.join("/Output")
print(datapath)
print(outputpath)
                

/Users/wasa/myPython/NLP/projects/WebTextProcessing/Data
/Users/wasa/myPython/NLP/projects/WebTextProcessing/Output


In [4]:
train_df = pd.read_csv(datapath + os.path.join(train_data), header=None)
train_df.columns = ["gender", "year", "kind", "opinion"]
test_df = pd.read_csv(datapath + os.path.join(test_data), header=None)
test_df.columns = ["gender", "year", "kind", "opinion"]
train_df.head(5)

,gender,year,kind,opinion
0,male,3rd,positive,慶應生でもあり、SFC生でもあること。就活などで慶應生に会うと親近感が湧いてくる。だが、SF...
1,female,1st,negative,外国語のインテンシブが普通の授業よりも１０分長く、全く集中が続かない。しかも休み時間は変わら...
2,male,3rd,positive,研究会は研究会等名前であるが実際はサークルのようなものであり、気軽に出入りすることができる、...
3,male,1st,negative,SFCは自然が豊かであるがために、虫がキャンパス内を横行する。朝はそんなに見かけることはない...
4,male,1st,positive,キャンパスの雰囲気がよいところ。郊外に位置し駅からも遠いため閑静であり、敷地は開放感があり緑...


In [5]:
pos_df = train_df[train_df['kind'] == 'positive'].reset_index()
neg_df = train_df[train_df['kind'] == 'negative'].reset_index()

In [6]:
pos_df.head(5)

,index,gender,year,kind,opinion
0,0,male,3rd,positive,慶應生でもあり、SFC生でもあること。就活などで慶應生に会うと親近感が湧いてくる。だが、SF...
1,2,male,3rd,positive,研究会は研究会等名前であるが実際はサークルのようなものであり、気軽に出入りすることができる、...
2,4,male,1st,positive,キャンパスの雰囲気がよいところ。郊外に位置し駅からも遠いため閑静であり、敷地は開放感があり緑...
3,6,male,3rd,positive,研究室が充実している。いろいろな施設があって学生証あればパソコンのある部屋に自由に出入りでき...
4,8,male,1st,positive,慶應義塾大学湘南藤沢キャンパスに通う生徒の個性が豊かであり、様々な分野の活動が見受けられる。...


In [7]:
neg_df.head(5)

,index,gender,year,kind,opinion
0,1,female,1st,negative,外国語のインテンシブが普通の授業よりも１０分長く、全く集中が続かない。しかも休み時間は変わら...
1,3,male,1st,negative,SFCは自然が豊かであるがために、虫がキャンパス内を横行する。朝はそんなに見かけることはない...
2,5,male,1st,negative,交通アクセスが不便です。湘南台駅からバス、自転車を使わないと学校にはつけない距離です。一度駅...
3,7,male,1st,negative,キャンパスが都心から遠いため、通学にかなりの時間がかかるところ。さらに、駅からも遠い上に公共...
4,9,male,1st,negative,駅からキャンパスまでバスで移動しなければならないということである。バスの本数は多い方だと思う...


In [8]:
nlp = spacy.load('ja_ginza_nopn')

AttributeError: module 'spacy' has no attribute 'load'

In [ ]:
def parse(df, key):
    res = np.array([])
    for i in range(len(df.index)):
        for sent in nlp(df.loc[i][key].strip()).sents:
            for token in sent:
                res = np.append(res, token.lemma_)
                
#                 print(token.i)
#                 print(token.orth_)
#                 print(token.lemma_)
#                 print(token.pos_)
#                 print(token._.pos_detail)
    return res

In [ ]:
pos_count = parse(pos_df, 'opinion')
pos_count_s = pd.Series(pos_count).value_counts()
word = pos_count_s.index
pos_count_df = pd.DataFrame({"word": word, "count" : pos_count_s.values})

In [ ]:
neg_count = parse(neg_df, 'opinion')
neg_count_s = pd.Series(neg_count).value_counts()
word = neg_count_s.index
neg_count_df = pd.DataFrame({"word": word, "count" : neg_count_s.values})

In [ ]:
all_count = parse(train_df, 'opinion')
all_count_s = pd.Series(all_count).value_counts()
word = all_count_s.index
all_count_df = pd.DataFrame({"word": word, "count" : all_count_s.values})

In [ ]:
pos_count_df.info()
pos_count_df.head(5)

In [ ]:
neg_count_df.info()
neg_count_df.head(5)

In [ ]:
all_count_df.info()
all_count_df.head(5)

In [ ]:
import math

In [ ]:
merged_df = pd.merge(all_count_df, pos_count_df, on="word", how="left")
merged_df = pd.merge(merged_df, neg_count_df, on="word", how="left").fillna(0)
merged_df.columns = ["word", "all", "pos", "neg"]
merged_df['pos_prob'] = merged_df['pos'] / merged_df['all']
merged_df['neg_prob'] = merged_df['neg'] / merged_df['all']
merged_df['pos_prob'] = merged_df['pos_prob'].apply(lambda x: 0.01 if x < 0.01 else x )
merged_df['neg_prob'] = merged_df['neg_prob'].apply(lambda x: 0.01 if x < 0.01 else x )
merged_df['log_pos_prob'] = merged_df['pos_prob'].apply(lambda x : math.log(x))
merged_df['log_neg_prob'] = merged_df['neg_prob'].apply(lambda x : math.log(x))

In [ ]:
merged_df

In [ ]:
pos_prob = len(pos_df.index) / len(train_df.index)
neg_prob = len(neg_df.index) / len(train_df.index)

In [ ]:
def classfication(prob_df, test_df):
    return_df = test_df.copy()
    return_df["predict_result"] = 0
    for i in range(len(test_df.index)):
        for sent in nlp(test_df.loc[i]['opinion'].strip()).sents:
            print("________")
            pos_probability = pos_prob
            neg_probability = neg_prob
            for token in sent:
                word = token.lemma_
                print(word)
                if word in prob_df['word'].values:
                    print('YES')
                    col = prob_df[prob_df['word'] == word]
                    print(col)
                    pos_probability += col['log_pos_prob'].values
                    neg_probability += col['log_neg_prob'].values
                    print(pos_probability)
                    print(neg_probability)
                    return_df.loc[i, 'predict_result'] = "positive" if pos_probability > neg_probability else "negative"
                    print(return_df.loc[i, 'predict_result'])
    return return_df
                
    

In [ ]:
result = classfication(merged_df, test_df)

In [ ]:
result

In [ ]:
result['YES'] = [1 if result.loc[i]['kind'] == result.loc[i]['predict_result'] else 0 for i in range(len(result.index))]

In [ ]:
result

In [ ]:
check = list(result['YES'].values)
check.count(1)

In [ ]:
print("accuracy is {}".format(check.count(1) / len(result.index)))

In [ ]:
test_df.head(5)